In [0]:
import pandas as pd
import numpy as np
import string
import random
from textblob.classifiers import NaiveBayesClassifier

In [0]:
#import Typo data
typo=pd.read_excel('RandomName_V1.xlsm',header=None,sheet_name='Copied',index_col=0)
typo[2]=typo[1]

In [0]:
typo.sample(3)

,1,2
0,,
5,Wilma Lee,Wilma Lee
9,Elbert Lawrence,Elbert Lawrence
1,Amber Woods,Amber Woods


In [0]:
#split data into three sets： A,B,C with 40%, 30%,30%
A, B, C = np.split(typo.sample(frac=1), [int(.4*len(typo)), int(.7*len(typo))])

In [0]:
#drop random letter in each rows
droplength= len(A)
for a in range(0,droplength):
    strforsample=str(A.iloc[a,1])
    wordlen=len(strforsample)
    wordloca=random.randint(0,wordlen-1)
    wordstrlist=list(strforsample)
    wordstrlist[wordloca]=''
    wordstr="".join(wordstrlist)
    A.iloc[a,1]=wordstr

In [0]:
#replace random letter in each rows
replacelength= len(B)
for a in range(0,replacelength):
    strforsample=str(B.iloc[a,1])
    wordlen=len(strforsample)
    wordloca=random.randint(0,wordlen-1)
    wordstrlist=list(strforsample)
    mylist = list(string.ascii_lowercase)
    wordstrlist[wordloca]=random.choice(mylist)
    wordstr="".join(wordstrlist)
    B.iloc[a,1]=wordstr

In [0]:
#insert random middlename in each rows
insertlength= len(C)
middlename=pd.read_excel('RandomName_V1.xlsm',header=None,sheet_name='middlenamelist')
middlename.dropna()
mylist= middlename[0].values.tolist()
for a in range(0,insertlength):
    randomtext=random.randint(2, 3)
    strforsample=str(C.iloc[a,1])
    wordlen=len(strforsample)
    wordstrlist=list(strforsample)
   
    #mylist = ["Rebekkah","Billie","Roxane","Nickola","Celie"]
 
    while True:
        randomcheck=random.randint(0, wordlen-1)
        if wordstrlist[randomcheck]==" ":
            #for times in range(0,randomtext):
            wordstrlist.insert(randomcheck+1, " ")
            wordstrlist.insert(randomcheck+1, random.choice(mylist))  
            break
       
    wordstr="".join(wordstrlist)
    C.iloc[a,1]=wordstr
C.head(10)

,1,2
0,,
8,Shane Joseph,Shane Lee Joseph
6,Tommy Ross,Tommy Grace Ross
8,Shane Joseph,Shane Grace Joseph
2,Tiffany Cain,Tiffany Jack Cain
6,Tommy Ross,Tommy Jay Ross
9,Elbert Lawrence,Elbert Charlotte Lawrence
5,Wilma Lee,Wilma Peter Lee
8,Shane Joseph,Shane Marie Joseph
3,Michelle Fuller,Michelle David Fuller


In [0]:
frames = [A, B, C]

typo_result = pd.concat(frames)

In [0]:
typo_result.sample(10)

,1,2
0,,
3,Michelle Fuller,MichelleFuller
4,Jared Glover,Jared Glove
8,Shane Joseph,Shane Joseh
10,Tim Clarke,Tim Clark
10,Tim Clarke,im Clarke
3,Michelle Fuller,Michell Fuller
10,Tim Clarke,Tim Clrke
2,Tiffany Cain,Tiffny Cain
1,Amber Woods,Aber Woods


In [0]:
#import typo_result data and select target columns.
typo=typo_result
typo[3]=typo.index
typo.columns = ['Original_Name','Typo','S.N.']
typo=typo[['Typo','Original_Name']]
typo.sample(10)

,Typo,Original_Name
0,,
8,Shane Joseh,Shane Joseph
9,Eldert Lawrence,Elbert Lawrence
4,Jared lover,Jared Glover
4,Jared qlover,Jared Glover
1,Amber Jade Woods,Amber Woods
9,Elbert sawrence,Elbert Lawrence
9,Elbert Jade Lawrence,Elbert Lawrence
9,Elbeat Lawrence,Elbert Lawrence
7,Pedro Lily Mills,Pedro Mills


In [0]:
#change to array.
typoarray=np.array(typo)
#select 80% as train data, 20% as test data. 
msk = np.random.rand(len(typoarray)) < 0.8
train_set =typoarray[msk]
test_set =typoarray[~msk]

In [0]:
#Using NaivebayesClassifier to classify typo into original name. 
typoclass = NaiveBayesClassifier(train_set)
typoclass.classify("Wilme Lee")

'Wilma Lee'

In [0]:
#Print out accuracy, 79.5 percent accuracy. Not bad. 
typoclass.accuracy(test_set )

0.9693877551020408

In [0]:
typoname=pd.read_excel('RandomName_V1.xlsm',header=None,sheet_name='Find difference -Paste')
typoname.dropna()
typoname[0]
typolist= typoname[0].values.tolist()
typolist

['Wilma Le',
 'Tiffamy Cain',
 'Shane C. Joseph',
 'Tommi Ross',
 'Shane Jose',
 'Jaraad Glover',
 'Amber Woeds',
 'Tommy Rass']

In [0]:
#Print out some samples and their prediction.
#typolist=["Willme Lee","Jared GloveR","Jered Glover","Tommy Rose","Shane Joeph","Shane J0seph"]
realnamelist=[]
for typo in typolist:
    realname=typoclass.classify(typo)
    print(realname)
    realnamelist.append(realname)

Wilma Lee
Tiffany Cain
Shane Joseph
Tommy Ross
Shane Joseph
Jared Glover
Amber Woods
Tommy Ross


In [0]:
typoclass.classify("Tommy Nick Rose")

'Tommy Ross'

In [0]:
#fuzzywuzzy

In [0]:
from fuzzywuzzy import process

str2Match = "Jared A. Glovery"

fuzzyname=pd.read_excel('RandomName_V1.xlsm',header=None,sheet_name='Original')
fuzzyname.dropna()

strOptions= fuzzyname[3].values.tolist()

In [0]:
fuzzyrealnamelist=[]
fuzzyrealnameprecent=[]
for typo in typolist:
    fuzzyrealname=process.extractOne(typo,strOptions)
    print(fuzzyrealname[0])
    fuzzyrealnamelist.append(fuzzyrealname[0])
    fuzzyrealnameprecent.append(fuzzyrealname[1])

Wilma Lee
Tiffany Cain
Shane Joseph
Tommy Ross
Shane Joseph
Jared Glover
Amber Woods
Tommy Ross


In [0]:
#change those prediction and realname into dictionary. 
correctionlist = {"typolist":typolist,"realnamelist":realnamelist, "fuzzyrealnamelist":fuzzyrealnamelist,"fuzzyrealnameprecent":fuzzyrealnameprecent}
correctiondf=pd.DataFrame.from_dict(correctionlist)
correctiondf.to_csv("typo_result_2.csv",index=True)